In [2]:
import os
import pandas as pd
 


In [200]:
# import data : returns multiple sheets
master_df = pd.read_excel('TC Input.xlsx',sheet_name=None)

In [290]:
# Combine sessions into one DF ; Attendees is a seperate sheet
brain_df = pd.DataFrame()

for k,v in master_df.items():
    if k == 'Attendees': # Create attendees df
        attendee_df = v
        attendee_df['dfname'] = k
    else: # Append session sheets. Save sheet name as dfname column
        df = v
        df['dfname'] = k
        brain_df = brain_df.append(v)

brain_df = brain_df.reset_index(drop=True).copy()

In [291]:
# Create datetime

# Convert dfname to date
brain_df['date'] = pd.to_datetime(brain_df['dfname']+'2021', format='%dth %b%Y')

# Wrangle data for session time. First confert to string, then concatenate strings then parsse to date
brain_df['Session Time'] = brain_df['Session Time'].astype('str')

times_df = brain_df['Session Time'].str.split(':',expand=True)

times_df = times_df.fillna('00')

times_df[0] = times_df[0].str.zfill(2)

times_df['time'] = times_df.agg(':'.join, axis=1)

In [292]:
# Join times table back to main table
brain_df = times_df[['time']].merge(brain_df,left_index=True, right_index=True).copy()

In [293]:
# Join date with time columns
brain_df['datetime'] = brain_df['date'].astype('str') + ' ' + brain_df['time']
brain_df['datetime'] = pd.to_datetime(brain_df['datetime'])

# General clean up, drop columns, reclassify, etc
brain_df = brain_df[['Session ID','Subject', 'Attendee IDs','datetime']].copy().reset_index(drop=True)
brain_df['Session ID'] = brain_df['Session ID'].astype('str')

In [296]:
# unpivot attendee ids so that we can join
ids_df = brain_df['Attendee IDs'].str.split(',',expand=True)
brain_df = brain_df.merge(ids_df,left_index=True,right_index=True)
brain_df = brain_df.melt(id_vars=['Session ID', 'Subject', 'Attendee IDs', 'datetime'], value_vars = [0,1,2,3,4]).reset_index(drop=True).copy()

In [298]:
# Clean up again ....
brain_df = brain_df[['Session ID', 'Subject', 'datetime', 'value']].rename(columns={'value':'attendee id'}).copy()

In [302]:
brain_df = brain_df.dropna(subset=['attendee id']).copy()

In [308]:
attendee_df['Attendee ID'] = attendee_df['Attendee ID'].astype('str')
mergedf = brain_df.merge(attendee_df,left_on='attendee id', right_on='Attendee ID',how='left')[['Session ID', 'Subject', 'datetime', 'Attendee']].copy()

In [313]:
brain_df.merge(attendee_df,left_on='attendee id', right_on='Attendee ID',how='left')

,Session ID,Subject,datetime,attendee id,Attendee ID,Attendee,dfname
0,1030,Desktop,2021-11-10 09:00:00,1,1,Kate,Attendees
1,1302,Prep,2021-11-10 09:30:00,5,5,Will,Attendees
2,1598,Prep,2021-11-10 10:00:00,1,1,Kate,Attendees
3,1727,Desktop,2021-11-10 10:30:00,8,8,Beth,Attendees
4,1760,Prep,2021-11-10 11:00:00,6,6,Carl,Attendees
...,...,...,...,...,...,...,...
225,8605,Community,2021-11-12 13:30:00,21,NaN,NaN,NaN
226,8710,Desktop,2021-11-12 14:00:00,22,NaN,NaN,NaN
227,9213,User Groups,2021-11-12 14:30:00,16,NaN,NaN,NaN
228,9570,Prep,2021-11-12 15:30:00,23,NaN,NaN,NaN


In [276]:
mergedf[mergedf['Attendee'] == 'Kate'].merge(mergedf[mergedf['Attendee'] == 'Will'],on=('Session ID'),how='outer')

,Session ID,Subject_x,datetime_x,Attendee_x,Subject_y,datetime_y,Attendee_y
0,1030,Desktop,2021-11-10 09:00:00,Kate,NaN,NaT,NaN
1,1598,Prep,2021-11-10 10:00:00,Kate,NaN,NaT,NaN
2,3127,Community,2021-11-10 16:00:00,Kate,NaN,NaT,NaN
3,5822,Server,2021-11-11 15:00:00,Kate,NaN,NaT,NaN
4,7840,DataDev,2021-11-12 11:00:00,Kate,NaN,NaT,NaN
5,8408,Server,2021-11-12 12:30:00,Kate,NaN,NaT,NaN
6,9213,User Groups,2021-11-12 14:30:00,Kate,NaN,NaT,NaN
7,9590,Desktop,2021-11-12 16:00:00,Kate,NaN,NaT,NaN
8,9727,Data+Women,2021-11-12 17:00:00,Kate,NaN,NaT,NaN
9,1302,NaN,NaT,NaN,Prep,2021-11-10 09:30:00,Will


In [303]:
brain_df

,Session ID,Subject,datetime,attendee id
0,1030,Desktop,2021-11-10 09:00:00,1
1,1302,Prep,2021-11-10 09:30:00,5
2,1598,Prep,2021-11-10 10:00:00,1
3,1727,Desktop,2021-11-10 10:30:00,8
4,1760,Prep,2021-11-10 11:00:00,6
...,...,...,...,...
241,8605,Community,2021-11-12 13:30:00,21
242,8710,Desktop,2021-11-12 14:00:00,22
243,9213,User Groups,2021-11-12 14:30:00,16
245,9570,Prep,2021-11-12 15:30:00,23


In [309]:
mergedf[mergedf['Session ID'] == '1727']

,Session ID,Subject,datetime,Attendee
3,1727,Desktop,2021-11-10 10:30:00,Beth
53,1727,Desktop,2021-11-10 10:30:00,NaN
103,1727,Desktop,2021-11-10 10:30:00,NaN
153,1727,Desktop,2021-11-10 10:30:00,NaN
203,1727,Desktop,2021-11-10 10:30:00,NaN


In [307]:
mergedf

,Session ID,Subject,datetime,Attendee
0,1030,Desktop,2021-11-10 09:00:00,Kate
1,1598,Prep,2021-11-10 10:00:00,Kate
2,3127,Community,2021-11-10 16:00:00,Kate
3,5822,Server,2021-11-11 15:00:00,Kate
4,7840,DataDev,2021-11-12 11:00:00,Kate
5,8408,Server,2021-11-12 12:30:00,Kate
6,9213,User Groups,2021-11-12 14:30:00,Kate
7,9590,Desktop,2021-11-12 16:00:00,Kate
8,9727,Data+Women,2021-11-12 17:00:00,Kate
9,1302,Prep,2021-11-10 09:30:00,Will


In [314]:
attendee_df

,Attendee ID,Attendee,dfname
0,1,Kate,Attendees
1,2,Tom,Attendees
2,3,Kelly,Attendees
3,4,Tim,Attendees
4,5,Will,Attendees
5,6,Carl,Attendees
6,7,Owen,Attendees
7,8,Beth,Attendees
8,9,Michael,Attendees
9,10,Rosario,Attendees


In [306]:
mergedf.to_csv('brain.csv')
os.startfile('brain.csv')

In [249]:
attendee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Attendee ID  30 non-null     int64 
 1   Attendee     30 non-null     object
 2   dfname       30 non-null     object
dtypes: int64(1), object(2)
memory usage: 848.0+ bytes
